In [38]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/pytho

In [15]:
import time
import pandas as pd
import torch
import random

mytime = time.localtime()

In [31]:
symptom = "pain"
time_period = "2 hours"
day_or_night = "night" if mytime.tm_hour < 12 else "day"   #("day" , "night")

general_questions = {
    'primary_symtom': f"Has the {symptom} been troubling you during the {day_or_night}? Please say Yes or No",
    'severity' :f"On a scale from 1 to 5, with 1 being very mild and 5 being extremely severe, how would you rate the severity of your {symptom}?",
    'duration' :f"Has the symptom been present for more than {time_period} continuously? Please say Yes or No",
    'continuous' : f"Can you tell me if  the {symptom} has been continuous? Please say Yes or No",
}

#resonance = f"On a scale from 1 to 5, with 1 indicating very rare occurrence and 5 indicating very frequent occurrence, how often does the {symptom} occur?"

In [37]:
general_questions['primary_symtom']

'Has the pain been troubling you during the day? Please say Yes or No'

In [5]:
custom_questions = []

In [6]:
look_out_for_the_words = {
    'primary_symtom': ["heart","chest pain","pain"],
    'severity' : ["4","5"],
    'duration' : [""],
    'continuous' :["continuous","intermittent"],
}

In [3]:
#Ask the doctor what symptom the assistant should ask about
symptom_list = []
time_period_list = []
time_of_day_to_ask_these_questions = []

while True:
  symptom = input('Which specific symptom the assistant should ask the patient about?\n')
  time_period =  input('At what duration of experiencing a symptom should we become concerned and consider it as potentially serious? \n')
  time_of_day_to_ask_these_questions_temp = []
  time_of_day_to_ask_these_questions_temp.append(input('At what time should the assistant ask the patient these questions?\n'))
  #time_of_day_to_ask_these_questions.append([time_of_day_to_ask_these_questions_temp])


  y_n = input('Is there any other time the assistant should ask the patient these questions? Please type y/n \n')
  if y_n == "y":
    time_of_day_to_ask_these_questions_temp.append(input('At what time should the assistant ask the patient these questions?\n'))
    #time_of_day_to_ask_these_questions.append(time_of_day_to_ask_these_questions_temp)

  symptom_list.append(symptom)
  time_period_list.append(time_period)
  time_of_day_to_ask_these_questions.append(time_of_day_to_ask_these_questions_temp)

  y_n = input('Do you want to add another symptom? Please type y/n \n')
  if y_n == "n":
    break


Which specific symptom the assistant should ask the patient about?
Heartburn
At what duration of experiencing a symptom should we become concerned and consider it as potentially serious? 
2 Hours
At what time should the assistant ask the patient these questions?
9
Is there any other time the assistant should ask the patient these questions? Please type y/n 
y
At what time should the assistant ask the patient these questions?
21
Do you want to add another symptom? Please type y/n 
y
Which specific symptom the assistant should ask the patient about?
Chest Pain
At what duration of experiencing a symptom should we become concerned and consider it as potentially serious? 
1 hour
At what time should the assistant ask the patient these questions?
8
Is there any other time the assistant should ask the patient these questions? Please type y/n 
n
Do you want to add another symptom? Please type y/n 
n


In [32]:
print(symptom_list)
print(time_period_list)
print(time_of_day_to_ask_these_questions)

['Heartburn', 'Chest Pain']
['2 Hours', '1 hour']
[['9', '21'], ['8']]


In [46]:
class Questions():
  def __init__(self,symptom,time_period,day_or_night):
    self.symptom = symptom
    self.time_period = time_period
    self.day_or_night = day_or_night

    self.primary_symtom = {"Question":f"Has the {self.symptom} been troubling you during the {self.day_or_night}? Please say Yes or No", "Answer": ""}
    self.severity = {"Question":f"On a scale from 1 to 5, with 1 being very mild and 5 being extremely severe, how would you rate the severity of your {self.symptom}?", "Answer": ""}
    self.duration = {"Question":f"Has the symptom been present for more than {self.time_period} continuously? Please say Yes or No", "Answer": ""}
    self.continuous = {"Question":f"Can you tell me if  the {self.symptom} has been continuous? Please say Yes or No", "Answer": ""}

def ask_questions(symptom_list, time_period_list, day_or_night):
    list_object = []
    for symptom,time_period in list(zip(symptom_list,time_period_list)):
      list_object.append(Questions(symptom,time_period,day_or_night))
    return list_object

In [52]:
list_obj = ask_questions(symptom_list, time_period_list, day_or_night)

print(list_obj[0].primary_symtom)
print(list_obj[1].primary_symtom)

{'Question': 'Has the Heartburn been troubling you during the night? Please say Yes or No', 'Answer': ''}
{'Question': 'Has the Chest Pain been troubling you during the night? Please say Yes or No', 'Answer': ''}


In [40]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

(…)bart-large-mnli/resolve/main/config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

(…)-mnli/resolve/main/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

(…)/bart-large-mnli/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)/bart-large-mnli/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)t-large-mnli/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [42]:
sequence_to_classify = "Yeah"
candidate_labels = ['yes', 'no']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'Yeah',
 'labels': ['yes', 'no'],
 'scores': [0.986284613609314, 0.01371534913778305]}

In [ ]:
#If patient tells the assistance they are having a perticular symptom